# Relative Value Score Investing 
In this notebook the goal is to find undervalued stocks by calculating the Relative Value Scores (RV scores) of the 50 lowest price to earnings ratios that are found in the stocks of the S&P 500. There are several ways of calculating the Relative values scores where it depends on what attributes on wants to consider. In this case the price to earnings ratio, price to book ratio, price to sales ratio, Evaluation-EBITDA ratio and Evaluation-Gross Profit ratio. Finding the RV Scores of these stocks gives an insight into which stocks might be undervalued which helps one to choose the value investing path och choosing these undervalued stocks.

In the table below one can see the all the columns that were needed to find the RV Score and the rest of the notebook highlights all the steps.



In [131]:
atr_dataframe.sort_values('RV Score', ascending= True)

,Symbol,Price,Number of Shares to Buy,Price to Earnings Ratio,PE Percentile,Price to Book Ratio,PB Percentile,Price to Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
2,EG,359.95,555,5.61,0.06,1.107067,0.12,1.002023,0.34,-705.808577,0.02,1.137074,0.02,-139.390483
29,WYNN,88.68,2255,10.65,0.6,-40.782437,0.02,1.569524,0.68,12.057803,0.9,7.363495,0.8,-1.828323
13,MO,52.38,3818,8.85,0.29,-20.095494,0.04,3.46981,0.96,7.586448,0.58,6.558740,0.64,1.273901
12,SYF,65.92,3033,8.56,0.26,1.15699,0.16,2.099403,0.82,-7.062270,0.06,2.324279,0.14,1.41568
17,GL,109.5,1826,9.28,0.36,2.579874,0.82,2.124886,0.84,-7.062270,0.06,2.664451,0.18,1.917388
47,DFS,174.6,1145,11.91,0.97,1.925384,0.54,2.900793,0.9,-7.062270,0.06,3.880878,0.38,2.710957
0,APA,22.09,9053,3.14,0.02,4.16235,0.92,1.334828,0.54,3.424851,0.12,2.747516,0.2,2.961909
8,CINF,144.05,1388,7.4,0.18,1.342637,0.22,1.622213,0.7,6.638092,0.46,1.618917,0.1,3.724372
28,PRU,119.06,1679,10.6,0.58,1.355089,0.24,0.698394,0.24,0.955245,0.1,5.450158,0.56,3.811777
22,BG,78.07,2561,9.89,0.46,1.38437,0.28,0.252298,0.04,4.523595,0.18,3.751456,0.36,3.960344


In [1]:
import numpy as np 
import pandas as pd
import requests
import xlsxwriter
import math
from scipy import stats

In [132]:
#Retrieving the stock symbols 
snpdata = pd.read_csv('sp500_companies.csv')
ticker_list = sorted(snpdata['Symbol'].tolist())

In [4]:
#Getting the API key and setting up the base url for this usecase
from secret import FMP_API_1
BASE_URL = "https://financialmodelingprep.com/api/v3/stock-price-change"

In [24]:
#Functions to save and load data from the API to json
import json
def save_to_json(data, filename):
    with open(filename, 'w') as json_file:
        json.dump(data, json_file, indent=4)
    print(f"Data saved to {filename}")

def load_from_json(filename):
    with open(filename, 'r') as json_file:
        data = json.load(json_file)
    print(f"Data retrieved from {filename}")
    return data     

In [35]:
current_data = load_from_json("ALL_Data_29th")

Data retrieved from ALL_Data_29th


In [135]:
#Retriveing the Price to Earnings ratio 
columns = ['Symbol', 'Price', 'Price to Earnings Ratio', 'Number of Shares to Buy' ]

new_dataframe = pd.DataFrame(columns=columns)

for idx in range(len(ticker_list)):

    new_row = pd.Series(
        [
            ticker_list[idx],                  
            current_data[idx]['price'],          
            current_data[idx]['pe'],          
            "NOS"        
        ],
        index=columns
    )
    new_dataframe = pd.concat([new_dataframe, new_row.to_frame().T], ignore_index=True)

new_dataframe = new_dataframe.sort_values("Price to Earnings Ratio")
new_dataframe = new_dataframe[new_dataframe["Price to Earnings Ratio"] > 0][:50]
new_dataframe


,Symbol,Price,Price to Earnings Ratio,Number of Shares to Buy
37,APA,22.09,3.14,NOS
190,FMC,49.01,4.02,NOS
155,EG,359.95,5.61,NOS
148,DVN,31.2,5.78,NOS
207,GM,54.28,5.79,NOS
5,ACGL,92.34,6.2,NOS
40,APTV,60.34,6.72,NOS
85,CE,68.76,6.85,NOS
93,CINF,144.05,7.4,NOS
70,BWA,32.19,7.97,NOS


In [45]:
#Deciding on number of share to buy
savings = 1000000
cut_per_stock = savings / len(new_dataframe.index)

for stock_idx in new_dataframe.index:
    # print(stock_idx)
    new_dataframe.loc[stock_idx, 'Number of Shares to Buy'] = math.floor(cut_per_stock/new_dataframe.loc[stock_idx, 'Price'])

new_dataframe


,Symbol,Price,Price-to-Earnings Ratio,Number of Shares to Buy
37,APA,22.09,3.14,905
190,FMC,49.01,4.02,408
155,EG,359.95,5.61,55
148,DVN,31.2,5.78,641
207,GM,54.28,5.79,368
5,ACGL,92.34,6.2,216
40,APTV,60.34,6.72,331
85,CE,68.76,6.85,290
93,CINF,144.05,7.4,138
70,BWA,32.19,7.97,621


In [52]:
#These are the links that help retrieve the needed valeus and ratios
ratios_l = 'https://financialmodelingprep.com/api/v3/ratios/'
enterprise_val = 'https://financialmodelingprep.com/api/v3/enterprise-values/'
ebitda_grossprofit = 'https://financialmodelingprep.com/api/v3/income-statement/'

#Function that retrieves all the data that is needed later on
def fetch_stock_data(symbols,api):
    ratios_list = []
    enterprise_list = []
    e_g_list = []
    for symbol in symbols:
        ratios = f"{ratios_l}/{symbol}?apikey={api}"
        r_response = requests.get(ratios)
        r_data = r_response.json()

        enterprise = f"{enterprise_val}/{symbol}?apikey={api}"
        e_response = requests.get(enterprise)
        e_data = e_response.json()

        eg = f"{ebitda_grossprofit}/{symbol}?apikey={api}"
        eg_response = requests.get(eg)
        eg_data = eg_response.json()
        
        ratios_list.append(r_data[0])
        enterprise_list.append(e_data[0])
        e_g_list.append(eg_data[0])

    return ratios_list, enterprise_list ,e_g_list

In [56]:
#Getting the symbols of the 50 chosen stocks
symbol_list = new_dataframe['Symbol'].tolist()

#Test case
r,e,eg = fetch_stock_data(["APA","GM"],FMP_API_1)
# print(r)
# print(e)
# print(eg)

In [57]:

r,e,eg = fetch_stock_data(symbol_list,FMP_API_1)

In [59]:
#Saving to json
save_to_json(r, "r")
save_to_json(e, "e")
save_to_json(eg, "eg")

Data saved to r
Data saved to e
Data saved to eg


In [114]:
r = load_from_json("r")
e = load_from_json("e")
eg = load_from_json("eg")

Data retrieved from r
Data retrieved from e
Data retrieved from eg


In [118]:
#Setting up the new data frame
columns = ['Symbol', 'Price', 'Number of Shares to Buy',
            'Price to Earnings Ratio',
            'PE Percentile',
            'Price to Book Ratio',
            'PB Percentile',
            'Price to Sales Ratio',
            'PS Percentile',
            'EV/EBITDA',
            'EV/EBITDA Percentile',
            'EV/GP',
            'EV/GP Percentile',
            'RV Score'
            ]
atr_dataframe = pd.DataFrame(columns=columns)

def stock_locator(symb,dict):
    for stock in dict:
        if stock['symbol'] == symb:
            return stock
        
for idx in range(len(symbol_list)):
    cur_stock = stock_locator(symbol_list[idx],current_data) 
    new_row = pd.Series(
        [
            symbol_list[idx],
            cur_stock['price'],
            'Holder',
            cur_stock['pe'],
            'Holder',
            r[idx]['priceToBookRatio'],
            'Holder',
            r[idx]['priceToSalesRatio'],
            'Holder',
            e[idx]['enterpriseValue']/eg[idx]['ebitda'] if eg[idx]['ebitda'] != 0 else np.NAN,
            'Holder',
            e[idx]['enterpriseValue']/eg[idx]['grossProfit'] if eg[idx]['grossProfit'] != 0 else np.NAN,
            'Holder',
            'Holder'
        ],
        index=columns
    )
    atr_dataframe = pd.concat([atr_dataframe, new_row.to_frame().T], ignore_index=True)

for column in ['Price to Earnings Ratio', 'Price to Book Ratio','Price to Sales Ratio', 'EV/EBITDA','EV/GP']:
    atr_dataframe[column].fillna(atr_dataframe[column].mean(),inplace= True)

atr_dataframe

C:\Users\hassa\AppData\Local\Temp\ipykernel_71340\2933492511.py:45: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  atr_dataframe[column].fillna(atr_dataframe[column].mean(),inplace= True)
C:\Users\hassa\AppData\Local\Temp\ipykernel_71340\2933492511.py:45: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  atr_datafra

,Symbol,Price,Number of Shares to Buy,Price to Earnings Ratio,PE Percentile,Price to Book Ratio,PB Percentile,Price to Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,APA,22.09,Holder,3.14,Holder,4.16235,Holder,1.334828,Holder,3.424851,Holder,2.747516,Holder,Holder
1,FMC,49.01,Holder,4.02,Holder,1.794386,Holder,1.764031,Holder,14.904167,Holder,6.399594,Holder,Holder
2,EG,359.95,Holder,5.61,Holder,1.107067,Holder,1.002023,Holder,-705.808577,Holder,1.137074,Holder,Holder
3,DVN,31.2,Holder,5.78,Holder,2.400025,Holder,1.911935,Holder,4.565293,Holder,6.645135,Holder,Holder
4,GM,54.28,Holder,5.79,Holder,0.762139,Holder,0.285116,Holder,6.628341,Holder,7.926431,Holder,Holder
5,ACGL,92.34,Holder,6.2,Holder,1.492037,Holder,2.00846,Holder,8.753328,Holder,2.385386,Holder,Holder
6,APTV,60.34,Holder,6.72,Holder,2.151478,Holder,1.239103,Holder,11.835936,Holder,8.675806,Holder,Holder
7,CE,68.76,Holder,6.85,Holder,2.384976,Holder,1.545874,Holder,11.172480,Holder,11.219694,Holder,Holder
8,CINF,144.05,Holder,7.4,Holder,1.342637,Holder,1.622213,Holder,6.638092,Holder,1.618917,Holder,Holder
9,BWA,32.19,Holder,7.97,Holder,1.413478,Holder,0.580205,Holder,6.363105,Holder,4.140479,Holder,Holder


In [121]:
#Calculatingall the Percentiles
per_dic = {
            'Price to Earnings Ratio':'PE Percentile',
            'Price to Book Ratio':'PB Percentile',
            'Price to Sales Ratio':'PS Percentile',
            'EV/EBITDA':'EV/EBITDA Percentile',
            'EV/GP':'EV/GP Percentile',
}

for row in atr_dataframe.index:
    for dic in per_dic.keys():
        atr_dataframe.loc[row, per_dic[dic]] = stats.percentileofscore(atr_dataframe[dic], atr_dataframe.loc[row, dic])/100


In [122]:
atr_dataframe

,Symbol,Price,Number of Shares to Buy,Price to Earnings Ratio,PE Percentile,Price to Book Ratio,PB Percentile,Price to Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,APA,22.09,Holder,3.14,0.02,4.16235,0.92,1.334828,0.54,3.424851,0.12,2.747516,0.2,Holder
1,FMC,49.01,Holder,4.02,0.04,1.794386,0.52,1.764031,0.74,14.904167,0.96,6.399594,0.62,Holder
2,EG,359.95,Holder,5.61,0.06,1.107067,0.12,1.002023,0.34,-705.808577,0.02,1.137074,0.02,Holder
3,DVN,31.2,Holder,5.78,0.08,2.400025,0.72,1.911935,0.78,4.565293,0.2,6.645135,0.66,Holder
4,GM,54.28,Holder,5.79,0.1,0.762139,0.06,0.285116,0.12,6.628341,0.44,7.926431,0.86,Holder
5,ACGL,92.34,Holder,6.2,0.12,1.492037,0.34,2.00846,0.8,8.753328,0.7,2.385386,0.16,Holder
6,APTV,60.34,Holder,6.72,0.14,2.151478,0.58,1.239103,0.5,11.835936,0.88,8.675806,0.9,Holder
7,CE,68.76,Holder,6.85,0.16,2.384976,0.68,1.545874,0.66,11.172480,0.86,11.219694,0.96,Holder
8,CINF,144.05,Holder,7.4,0.18,1.342637,0.22,1.622213,0.7,6.638092,0.46,1.618917,0.1,Holder
9,BWA,32.19,Holder,7.97,0.2,1.413478,0.3,0.580205,0.2,6.363105,0.42,4.140479,0.44,Holder


In [127]:
#Calculating the RV Score
for row in atr_dataframe.index[:2]:
    mean_list = []
    for dic in per_dic.keys():

        mean_list.append(atr_dataframe.loc[row, dic])
    atr_dataframe.loc[row, 'RV Score'] = np.mean(mean_list)

atr_dataframe

,Symbol,Price,Number of Shares to Buy,Price to Earnings Ratio,PE Percentile,Price to Book Ratio,PB Percentile,Price to Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,APA,22.09,Holder,3.14,0.02,4.16235,0.92,1.334828,0.54,3.424851,0.12,2.747516,0.2,2.961909
1,FMC,49.01,Holder,4.02,0.04,1.794386,0.52,1.764031,0.74,14.904167,0.96,6.399594,0.62,5.776435
2,EG,359.95,Holder,5.61,0.06,1.107067,0.12,1.002023,0.34,-705.808577,0.02,1.137074,0.02,-139.390483
3,DVN,31.2,Holder,5.78,0.08,2.400025,0.72,1.911935,0.78,4.565293,0.2,6.645135,0.66,4.260477
4,GM,54.28,Holder,5.79,0.1,0.762139,0.06,0.285116,0.12,6.628341,0.44,7.926431,0.86,4.278405
5,ACGL,92.34,Holder,6.2,0.12,1.492037,0.34,2.00846,0.8,8.753328,0.7,2.385386,0.16,4.167842
6,APTV,60.34,Holder,6.72,0.14,2.151478,0.58,1.239103,0.5,11.835936,0.88,8.675806,0.9,6.124465
7,CE,68.76,Holder,6.85,0.16,2.384976,0.68,1.545874,0.66,11.172480,0.86,11.219694,0.96,6.634605
8,CINF,144.05,Holder,7.4,0.18,1.342637,0.22,1.622213,0.7,6.638092,0.46,1.618917,0.1,3.724372
9,BWA,32.19,Holder,7.97,0.2,1.413478,0.3,0.580205,0.2,6.363105,0.42,4.140479,0.44,4.093453


In [128]:
atr_dataframe.sort_values('RV Score', ascending= True)

,Symbol,Price,Number of Shares to Buy,Price to Earnings Ratio,PE Percentile,Price to Book Ratio,PB Percentile,Price to Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
2,EG,359.95,Holder,5.61,0.06,1.107067,0.12,1.002023,0.34,-705.808577,0.02,1.137074,0.02,-139.390483
29,WYNN,88.68,Holder,10.65,0.6,-40.782437,0.02,1.569524,0.68,12.057803,0.9,7.363495,0.8,-1.828323
13,MO,52.38,Holder,8.85,0.29,-20.095494,0.04,3.46981,0.96,7.586448,0.58,6.558740,0.64,1.273901
12,SYF,65.92,Holder,8.56,0.26,1.15699,0.16,2.099403,0.82,-7.062270,0.06,2.324279,0.14,1.41568
17,GL,109.5,Holder,9.28,0.36,2.579874,0.82,2.124886,0.84,-7.062270,0.06,2.664451,0.18,1.917388
47,DFS,174.6,Holder,11.91,0.97,1.925384,0.54,2.900793,0.9,-7.062270,0.06,3.880878,0.38,2.710957
0,APA,22.09,Holder,3.14,0.02,4.16235,0.92,1.334828,0.54,3.424851,0.12,2.747516,0.2,2.961909
8,CINF,144.05,Holder,7.4,0.18,1.342637,0.22,1.622213,0.7,6.638092,0.46,1.618917,0.1,3.724372
28,PRU,119.06,Holder,10.6,0.58,1.355089,0.24,0.698394,0.24,0.955245,0.1,5.450158,0.56,3.811777
22,BG,78.07,Holder,9.89,0.46,1.38437,0.28,0.252298,0.04,4.523595,0.18,3.751456,0.36,3.960344


In [130]:
savings = 10000000
cut_per_stock = savings / len(new_dataframe.index)


for stock in atr_dataframe.index:
    atr_dataframe.loc[stock, 'Number of Shares to Buy'] = math.floor(cut_per_stock/atr_dataframe.loc[stock,'Price'])

atr_dataframe

,Symbol,Price,Number of Shares to Buy,Price to Earnings Ratio,PE Percentile,Price to Book Ratio,PB Percentile,Price to Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,APA,22.09,9053,3.14,0.02,4.16235,0.92,1.334828,0.54,3.424851,0.12,2.747516,0.2,2.961909
1,FMC,49.01,4080,4.02,0.04,1.794386,0.52,1.764031,0.74,14.904167,0.96,6.399594,0.62,5.776435
2,EG,359.95,555,5.61,0.06,1.107067,0.12,1.002023,0.34,-705.808577,0.02,1.137074,0.02,-139.390483
3,DVN,31.2,6410,5.78,0.08,2.400025,0.72,1.911935,0.78,4.565293,0.2,6.645135,0.66,4.260477
4,GM,54.28,3684,5.79,0.1,0.762139,0.06,0.285116,0.12,6.628341,0.44,7.926431,0.86,4.278405
5,ACGL,92.34,2165,6.2,0.12,1.492037,0.34,2.00846,0.8,8.753328,0.7,2.385386,0.16,4.167842
6,APTV,60.34,3314,6.72,0.14,2.151478,0.58,1.239103,0.5,11.835936,0.88,8.675806,0.9,6.124465
7,CE,68.76,2908,6.85,0.16,2.384976,0.68,1.545874,0.66,11.172480,0.86,11.219694,0.96,6.634605
8,CINF,144.05,1388,7.4,0.18,1.342637,0.22,1.622213,0.7,6.638092,0.46,1.618917,0.1,3.724372
9,BWA,32.19,6213,7.97,0.2,1.413478,0.3,0.580205,0.2,6.363105,0.42,4.140479,0.44,4.093453
